In [1]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np

In [2]:
class skyDataset(Dataset):
    def __init__(self, dataset_dim, data_path, label_path):
        super().__init__()
        self.image_w = 175
        self.data = np.memmap(data_path, dtype='float32', mode='r', shape=(dataset_dim, 3, self.image_w, self.image_w))
        self.label = np.memmap(label_path, dtype='uint8', mode='r', shape=(dataset_dim,))
        self.size = dataset_dim

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        image = self.data[idx]
        image = torch.from_numpy(np.array(image))

        label = torch.tensor(self.label[idx], dtype=torch.float32)

        return image, label

In [3]:
class skyModel(nn.Module):
    def __init__(self):
        super(skyModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 3, 3, 2)  # 3 out channel, 3x3 con stride 2
        self.conv2 = nn.Conv2d(3, 3, 5, 2)  # 3 out channel, 5x5 con stride 2
        self.pooling = nn.MaxPool2d(2, 2)  # 2x2 con stride 2
        self.relu = nn.ReLU()
        self.linear = nn.Linear(1323, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pooling(x)

        x = torch.flatten(x, 1)
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x


def eval_acc(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y_true in data_loader:

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)
            y_pred = torch.round(y_pred)

            total += y_true.size(0)
            c = (y_pred == y_true).sum().item()
            correct += c

    return correct / total


In [4]:
train_data = os.path.join(".","dataset", "train_data.npm")
train_label = os.path.join(".","dataset", "train_label.npm")
test_data = os.path.join(".","dataset", "test_data.npm")
test_label = os.path.join(".","dataset", "test_label.npm")

train_dataset = skyDataset(12000, train_data, train_label)
test_dataset = skyDataset(1564, test_data, test_label)

dl_train = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
dl_test = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [5]:
learning_rate = 0.01
epoch_num = 5

model = skyModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_values = []
train_accs = []
test_accs = []

for i in range(epoch_num):
    for j, (x, y_true) in enumerate(dl_train):
        y_pred = model(x)
        y_pred = torch.squeeze(y_pred)

        loss = loss_fn(y_pred, y_true)
        l = loss.item()
        loss_values.append(l)
        #print("Epoch:", i, "Batch:", j, "Loss:", l)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if j%50 == 0:
            train_acc = eval_acc(model, dl_train)
            test_acc = eval_acc(model, dl_test)
            print("Epoch:", i, "Batch:", j)
            print("Train acc:", train_acc)
            print("Test acc:", test_acc , "\n")
            train_accs.append(train_acc)
            test_accs.append(test_accs)


torch.save(model.state_dict(), "skyModel.bin")
print(loss_values)
print(train_accs)
print(test_accs)

Epoch: 0 Batch: 0
Train acc: 0.6071666666666666
Test acc: 0.5671355498721228 

Epoch: 0 Batch: 50
Train acc: 0.8645833333333334
Test acc: 0.8037084398976982 

Epoch: 0 Batch: 100
Train acc: 0.8939166666666667
Test acc: 0.8254475703324808 

Epoch: 0 Batch: 150
Train acc: 0.9161666666666667
Test acc: 0.8363171355498721 

Epoch: 0 Batch: 200
Train acc: 0.91675
Test acc: 0.8209718670076727 

Epoch: 0 Batch: 250
Train acc: 0.9253333333333333
Test acc: 0.8203324808184144 

Epoch: 0 Batch: 300
Train acc: 0.9340833333333334
Test acc: 0.8267263427109974 

Epoch: 0 Batch: 350
Train acc: 0.9306666666666666
Test acc: 0.8113810741687979 

Epoch: 1 Batch: 0
Train acc: 0.9406666666666667
Test acc: 0.8273657289002557 

Epoch: 1 Batch: 50
Train acc: 0.939
Test acc: 0.8267263427109974 

Epoch: 1 Batch: 100
Train acc: 0.9366666666666666
Test acc: 0.8113810741687979 

Epoch: 1 Batch: 150
Train acc: 0.945
Test acc: 0.8235294117647058 

Epoch: 1 Batch: 200
Train acc: 0.938
Test acc: 0.8101023017902813 

Epo

KeyboardInterrupt: 